In [1]:
import sys
sys.path.insert(1, './env')
sys.path.insert(1, './agents')

import gym
import pylab
import random
import strategy_imitation, sarsa, ddqn, random_agent, a2c
import aa_gun
import numpy as np
from collections import deque
import keras
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization

import matplotlib.pyplot as plt
import pandas as pd

Using TensorFlow backend.


In [2]:
#массовый тест моделей

In [ ]:
#Проверь на зенитке, на cartpole и на mountain car

score_dict_full={'random':[],'ddqn':[],'a2c':[],'sarsa':[]}
border_med = 100
EPISODES=100
score_dict_med={'random':[],'ddqn':[],'a2c':[],'sarsa':[]}#c border_med по... такты. Надо, чтобы проверить быстроту обучения

agent_list=[random_agent.randomAgent,a2c.A2CAgent,ddqn.DoubleDQNAgent,sarsa.SarsaAgent]
for ag_num in range(3,4):
    if ag_num==0:
        name='random'
    if ag_num==1:
        name='a2c'
    if ag_num==2:
        name='ddqn' 
    if ag_num==3:
        name='sarsa'
    
    for estimation in range(3):
        print('_____',name,estimation,pd.Timestamp.now())
        #здесь весь код от инициализации модели до выдачи scores. Но без рендера.
        # In case of CartPole-v1, maximum length of episode is 500
        
        #env = aa_gun.AA_gun_simple0_env()
        env = gym.make('Seaquest-ramNoFrameskip-v0')
        #env=CartPoleEnv9()
        # get size of state and action from environment
        state_size = env.observation_space.shape[0]
        action_size = env.action_space.n

        #agent = DoubleDQNAgent(state_size, action_size)
        agent = agent_list[ag_num](state_size, action_size)
        agent.train_start=100
        agent.train_start=7000
        agent.epsilon_decay=0.9999
        agent.render=False

        scores, episodes = [], []
        reward_lst = []
        s_list=[]
        a_list=[]

        for e in range(EPISODES):
            done = False
            score = 0
            state = env.reset()
            state = np.reshape(state, [1, state_size])

            while not done:
                #if (e in range(2,7)) or (e in range(20,25)) or (e in range(100,103)) or (e in range(200,202)) or (e in range(300,306)) or (e in range(400,406)) or (e in range(500,506)) or (e in range(600,604)):
                #    if agent.render:
                #        env.render()

                # get action for the current state and go one step in environment
                action = agent.get_action(state)
                next_state, reward, done, info = env.step(action)
                next_state = np.reshape(next_state, [1, state_size])
                # if an action make the episode end, then gives penalty of -100

                
                # save the sample <s, a, r, s'> to the replay memory
                agent.append_sample(state, action, reward, next_state, done)
                #if next_state[0,11]!=reward:
                #    print('state[13]!=reward',state[0,11],reward)
                #
                s_list.append(state)
                a_list.append(action)
                reward_lst.append(reward)
                #

                # every time step do the training
                agent.train_model()
                score += reward
                state = next_state

                if done:
                    # every episode update the target model to be same with model
                    agent.update_target_model()

                    # every episode, plot the play time
                    scores.append(score)
                    episodes.append(e)
                    pylab.plot(episodes, scores, 'b')
                    #pylab.savefig("./save_graph/aa_gun_dqn.png")
                    print("episode:", e, "  score:", score, "  memory length:",
                          len(agent.memory), "  epsilon:", agent.epsilon)



            # save the model
            #if e % 50 == 0:
            #    agent.model.save_weights("./save_model/aa_gun_dqn.h5")



        #и первые 3000 тактов - это рандом
        #Ходов так 50
        score_dict_full[name].append(np.mean(scores))
        score_dict_med[name].append(np.mean(scores[border_med:]))
        import pickle
        f=open('score_dict.pkl','wb')
        pickle.dump([score_dict_full,score_dict_med],f)
        plt.show()

_____ sarsa 0 2020-01-23 16:29:07.831197
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 100)               12900     
_________________________________________________________________
batch_normalization_13 (Batc (None, 100)               400       
_________________________________________________________________
dropout_13 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 100)               10100     
_________________________________________________________________
batch_normalization_14 (Batc (None, 100)               400       
_________________________________________________________________
dropout_14 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_21 (Den

Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 51.0295
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 50.8916
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 53.4914
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 51.6987
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 50.4417
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 48.9199
Epoch 2/2
500/500 [==============================] - 0s 15us/step - loss: 48.7541
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 49.5976
Epoch 2/2
500/500 [==============================] - 0s 15us/step - loss: 48.9083
Epoch 1/2
500/500 [==============================] - 0s 16us/step - loss: 49.5963
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 48.8475
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 48.0862
Epoch 2/2
500/50

Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 65.6003
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 68.1190
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 62.3198
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 65.7649
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 65.7151
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 65.4845
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 64.9608
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 62.2268
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 62.0308
Epoch 2/2
500/500 [==============================] - 0s 15us/step - loss: 61.1972
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 62.0793
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 60.1585
Epoch 1/2
500/50

Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 66.8354
Epoch 1/2
500/500 [==============================] - 0s 12us/step - loss: 65.7758
Epoch 2/2
500/500 [==============================] - 0s 12us/step - loss: 68.5577
Epoch 1/2
500/500 [==============================] - 0s 12us/step - loss: 66.3668
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 67.6190
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 68.3846
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 64.6035
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 64.6609
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 62.2202
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 63.6280
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 63.1932
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 63.6585
Epoch 2/2
500/50

Epoch 1/2
500/500 [==============================] - 0s 12us/step - loss: 41.7162
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 40.7167
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 36.6356
Epoch 2/2
500/500 [==============================] - 0s 12us/step - loss: 40.5332
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 38.4301
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 41.3117
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 38.9023
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 31.8734
Epoch 1/2
500/500 [==============================] - 0s 12us/step - loss: 38.6619
Epoch 2/2
500/500 [==============================] - 0s 12us/step - loss: 36.0319
Epoch 1/2
500/500 [==============================] - 0s 12us/step - loss: 41.5121
Epoch 2/2
500/500 [==============================] - 0s 12us/step - loss: 36.5653
Epoch 1/2
500/50

...

Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.6427
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.4914
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.5132
Epoch 2/2
500/500 [==============================] - 0s 12us/step - loss: 3.4456
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.7106
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.5019
delta_r 2.280933660956428 r 8.09643542877356
Epoch 1/2
500/500 [==============================] - 0s 15us/step - loss: 10.6895
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 10.7872
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 4.3878
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 4.4050
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 4.2513
Epoch 2/2
500/500 [==============================] - 0s 14us/s

Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.4573
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.5655
Epoch 2/2
500/500 [==============================] - 0s 12us/step - loss: 3.4191
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.4160
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.4275
Epoch 1/2
500/500 [==============================] - 0s 12us/step - loss: 3.4175
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.3684
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.2675
Epoch 2/2
500/500 [==============================] - 0s 12us/step - loss: 3.4421
Epoch 1/2
500/500 [==============================] - 0s 12us/step - loss: 3.3859
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.2944
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.2040
Epoch 2/2
500/500 [=========

Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.0394
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.0387
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 2.9670
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 2.9263
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 3.0230
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 2.9509
delta_r 1.935054748546418 r 7.768342663342355
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 8.8848
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 10.2343
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.1256
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.2354
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.2353
Epoch 2/2
500/500 [==============================] - 0s 13us/s

Epoch 2/2
500/500 [==============================] - 0s 12us/step - loss: 3.3180
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.1838
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.1029
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 3.1719
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 3.1839
Epoch 1/2
500/500 [==============================] - 0s 15us/step - loss: 3.2002
Epoch 2/2
500/500 [==============================] - 0s 14us/step - loss: 2.9352
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.0587
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.1980
Epoch 1/2
500/500 [==============================] - 0s 14us/step - loss: 2.9679
Epoch 2/2
500/500 [==============================] - 0s 13us/step - loss: 3.1809
Epoch 1/2
500/500 [==============================] - 0s 13us/step - loss: 3.1756
Epoch 2/2
500/500 [=========

Epoch 1/1
6000/6000 [==============================] - 0s 5us/step - loss: 6.9624
Epoch 1/1
6000/6000 [==============================] - 0s 6us/step - loss: 3.1747
delta_r 2.1682114907121446 r 7.428773156589693
episode: 20   score: 120.0   memory length: 10000   epsilon: 0.009999399224421553
Epoch 1/1
6000/6000 [==============================] - 0s 5us/step - loss: 7.7987
Epoch 1/1
6000/6000 [==============================] - 0s 6us/step - loss: 2.6500
delta_r 2.2360079016205576 r 7.399503869945344
episode: 21   score: 120.0   memory length: 10000   epsilon: 0.009999399224421553
Epoch 1/1
6000/6000 [==============================] - 0s 5us/step - loss: 5.7552
Epoch 1/1
6000/6000 [==============================] - 0s 6us/step - loss: 2.3450
delta_r 2.394535521071113 r 6.803995690572631
episode: 22   score: 100.0   memory length: 10000   epsilon: 0.009999399224421553
Epoch 1/1
6000/6000 [==============================] - 0s 11us/step - loss: 5.0441
Epoch 1/1
6000/6000 [==================

In [ ]:
score_dict_full